# Intro to Parquet

In [4]:
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd

## Save & Write

In [19]:
# get iris dataset
iris = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')
iris.columns

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width',
       'species'],
      dtype='object')

In [21]:
iris.species.unique()

array(['setosa', 'versicolor', 'virginica'], dtype=object)

In [22]:
# let;s separate virginica
virginica = iris[iris.species == 'virginica']
others = iris[iris.species != 'virginica']

virginica.shape, others.shape

((50, 5), (100, 5))

In [35]:
# save separately as parquet
virginica.to_parquet('tmp/virginica.parquet', engine='pyarrow', index = False)
others.to_parquet('tmp/others.parquet', engine='pyarrow', index = False)

## Useful Operations with Parquet

### Combining Tables

In [30]:
others_pq = pq.read_table('tmp/others.parquet')
others_pq.to_pandas().head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [31]:
virginica_pq = pq.read_table('tmp/virginica.parquet')
virginica_pq.to_pandas().head()

,sepal_length,sepal_width,petal_length,petal_width,species
100,6.3,3.3,6.0,2.5,virginica
101,5.8,2.7,5.1,1.9,virginica
102,7.1,3.0,5.9,2.1,virginica
103,6.3,2.9,5.6,1.8,virginica
104,6.5,3.0,5.8,2.2,virginica


In [33]:
to_merge = [
    others_pq, virginica_pq
]
merged_table = pa.concat_tables(to_merge)
merged_table.to_pandas().drop_duplicates(subset = 'species')

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
100,6.3,3.3,6.0,2.5,virginica


### Filtering data while reading (predicate pushdown)

In [36]:
import pyarrow.parquet as pq

table = pq.read_table('tmp/others.parquet', filters=[
    ('species', '=', 'setosa'), ('petal_length', '>', 1.5)
])

table.to_pandas().head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.4,3.9,1.7,0.4,setosa
1,4.8,3.4,1.6,0.2,setosa
2,5.7,3.8,1.7,0.3,setosa
3,5.4,3.4,1.7,0.2,setosa
4,5.1,3.3,1.7,0.5,setosa


### Merge multiple parquet files

In [41]:
list_files = [
    'tmp/others.parquet', 'tmp/virginica.parquet'
]
tables = [
    pq.read_table(f) for f in list_files
]
merged_table = pa.concat_tables(tables)
merged_table.to_pandas().head()
pq.write_table(merged_table, "tmp/iris.parquet")

### Partition parquet files for faster queries

In [50]:
iris
iris.to_parquet("tmp/partitioned", partition_cols = ['species'])